In [1]:
import pandas as pd
import graphlab as gl
from random import random
gl.canvas.set_target('ipynb')
%matplotlib inline

[INFO] Start server at: ipc:///tmp/graphlab_server-11761 - Server binary: /Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/graphlab/unity_server - Server log: /tmp/graphlab_server_1425935061.log
[INFO] GraphLab Server Version: 1.3.0


In [2]:
# clean pairs data
def clean_pair_date(sf):
    sf.rename({'X1': 'lender_id', 'X2': 'loan_id'})
    sf['loan_id'] = sf['loan_id'].astype(str)
    sf['lender_id'] = sf['lender_id'].astype(str)
    return sf


# clean loan data
def clean_loan_data(df):
    # drop columns
    df = df.drop(['paid_date', 'planned_expiration_date', 'languages'], axis=1)

    # drop duplicates
    df = df.drop_duplicates('id')

    # drop duplicate header
    df = df[df['activity'] != 'activity']

    # drop nas
    df = df.dropna(subset=['earliest_scheduled_payment', 'last_scheduled_payment',
                           'repayment_interval', 'posted_date',
                           'status', 'repayment_term', 'use'], how='any')

    # fill paid_amount's na with zero
    df['paid_amount'] = df['paid_amount'].fillna(0)

    # fill genders
    df['gender'] = df['gender'].map(lambda x: 'M' if random() <= 0.39 else 'F')
    df['gender'] = df['gender'].map(lambda x: 1 if x == 'F' else 'M')

    # fill null descriptions with empty string
    df['descriptions'] = df['descriptions'].fillna(0)

    # binaralize bonus credit
    df['bonus_credit_eligibility'] = df['bonus_credit_eligibility'].map(lambda x: 1 if x == 'True' else 0)

    # convert some columns to datetime
    df['earliest_scheduled_payment'] = df['earliest_scheduled_payment'].map(
        lambda x: pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%SZ'))
    df['last_scheduled_payment'] = df['last_scheduled_payment'].map(
        lambda x: pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%SZ'))
    df['posted_date'] = df['posted_date'].map(
        lambda x: pd.to_datetime(x, format='%Y-%m-%dT%H:%M:%SZ'))

    # convert some columns to int
    df['lender_count'] = df['lender_count'].astype('int64')
    df['loan_amount'] = df['loan_amount'].astype('int64')
    df['paid_amount'] = df['paid_amount'].astype('float64').astype('int64')
    df['repayment_term'] = df['repayment_term'].astype('float64')

    return df


def drop_unexsiting_loan_ids(sf, df):
    loan_ids_in_pairs = sorted(list(sf['loan_id'].unique()))
    loan_ids_in_loans = sorted(list(df['id'].values))
    loan_ids_intersection = set(loan_ids_in_loans) & set(loan_ids_in_pairs)
    # drop useless loan_ids in sf
    sf['loan_id'] = sf['loan_id'].apply(lambda x: x if x in loan_ids_intersection else None)
    sf = sf.dropna('loan_id')
    # drop useless loan_ids in df
    df['id'] = df['id'].map(lambda x: x if x in loan_ids_intersection else None)
    df = df.dropna()
    return sf, df

In [3]:
df = pd.read_csv('data/loans.csv', delimiter=',')
df = clean_loan_data(df)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 824505 entries, 0 to 842841
Data columns (total 21 columns):
activity                      824505 non-null object
bonus_credit_eligibility      824505 non-null int64
id                            824505 non-null object
lender_count                  824505 non-null int64
loan_amount                   824505 non-null int64
paid_amount                   824505 non-null int64
partner_id                    824505 non-null object
posted_date                   824505 non-null datetime64[ns]
sector                        824505 non-null object
status                        824505 non-null object
use                           824505 non-null object
gender                        824505 non-null object
family                        824505 non-null object
descriptions                  824505 non-null object
image_id                      824505 non-null object
image_template_id             824505 non-null object
country                       824505 

## Get features!!!!!!!
- bonus credit eligibility
- loan_amount
- posted_date =>
 - 4 seasons: dummy variable in {0, 1, 2, 3}
- Tfidf of use
- gender
- family
- country
- repayment term
- repayment interval => dummy variable in {0, 1, 2}

In [9]:
raw_features = df[['bonus_credit_eligibility', 'loan_amount', 'posted_date',
                   'use', 'gender', 'family', 'country', 'repayment_term',
                   'repayment_interval', 'id']]

In [6]:
df['repayment_term'].describe()

count    824505.000000
mean         12.830616
std           7.463898
min           2.000000
25%           8.000000
50%          12.000000
75%          14.000000
max         195.000000
Name: repayment_term, dtype: float64

In [8]:
df['repayment_interval'].value_counts()

Monthly           568528
Irregularly       187488
At end of term     68489
dtype: int64